In [18]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

In [19]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

In [20]:
import minsearch

In [21]:
import json
with open('documents.json', 'rt') as f_in:
    docs_raw=json.load(f_in)

In [22]:
docs_raw[1].keys()
docs_raw[1]['documents'][1].keys()

dict_keys(['text', 'section', 'question'])

In [23]:
#docs_raw is a list of 3 dictionaries
print(len(docs_raw))
# each dictionary has two keys
print(docs_raw[0].keys())
# course key has one item
print(docs_raw[0]['course'])
# documents key has a list dictionaries which add up to 435
print(len(docs_raw[0]['documents']))
# each of those dictionaries has three keys namely
print(docs_raw[0]['documents'][0].keys())

3
dict_keys(['course', 'documents'])
data-engineering-zoomcamp
435
dict_keys(['text', 'section', 'question'])


In [24]:
len(docs_raw)
#docs_raw[0]
#len(docs_row[0])

3

In [25]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [26]:
(documents[0].keys())

dict_keys(['text', 'section', 'question', 'course'])

In [27]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]  
)


In [28]:
q = 'the course has already started, can I still enroll?'

In [29]:
index.fit(documents)

In [30]:
boost ={'question':3.0, 'section':0.5}
results= index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5

    
)

In [31]:
#print(results)
results


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [32]:
from openai import OpenAI
client = OpenAI()
#client = OpenAI()

In [33]:
import os
os.environ
envir=os.environ
#envir


In [1]:
# if 'OPENAI_API_KEY' in envir:
#     print(envir['OPENAI_API_KEY'])
# else:
#     print(False)

In [35]:
response = client.chat.completions.create(
    #model='gpt-3.5-turbo',
    model='gpt-4',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It's possible that you can still enroll, but it's dependent on the specific rules and policies of the course you're interested in. Some courses allow late registration, while others do not. We recommend contacting the course administrator or institution offering the course to find out for sure."

In [36]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results
query=search(q)
query

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [37]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [38]:
prompt=build_prompt(query=q, search_results=results)
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related 

In [39]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [40]:
llm(prompt)

"Yes, you can still enroll in the course after it has started. Even if you don't register, you'll be eligible to submit the homeworks. Just be aware of the deadlines for the final projects and plan accordingly to avoid leaving everything to the last minute."

In [41]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [42]:
rag(query)

'To run Kafka using Java in the terminal, navigate to your project directory and execute the following command:\n\n```shell\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your JAR file.'

In [43]:
rag('the course has started. Can I still enroll?')

'Yes, you can still enroll in the course even after it has started. You are also eligible to submit the homework assignments. However, note that there will be deadlines for turning in the final projects, so be sure not to leave everything for the last minute.'

In [44]:
rag('Tell me about Kilimanjaro Mountain in Kenya')

'I don’t have information regarding Kilimanjaro Mountain in Kenya. You might want to look into other resources or databases for information on that topic.'

In [45]:
def aigenerate(prompt):
    answer = llm(prompt)
    return answer

In [46]:
aigenerate('Tell me about Kilimanjaro Mountain in Kenya')

"Kilimanjaro is actually located in Tanzania, not Kenya. It is the highest mountain in Africa, standing at 5,895 meters (19,341 feet) above sea level. Mount Kilimanjaro is composed of three distinct volcanic cones: Kibo, Mawenzi, and Shira. Of these, Kibo is the tallest and where Uhuru Peak, the highest summit on Kilimanjaro, is found.\n\nHere's a more detailed look at Mount Kilimanjaro:\n\n### Geographical Context\n- **Location:** Kilimanjaro is in the northeastern part of Tanzania, near the border with Kenya.\n- **Ecosystem:** The mountain has a variety of ecological zones, including cultivated land, rainforest, heath, alpine desert, and the arctic summit.\n- **National Park:** Kilimanjaro National Park protects the area around the mountain and has been designated a UNESCO World Heritage Site.\n\n### Climbing Kilimanjaro\n- **Routes:** There are several routes to ascend Kilimanjaro, each offering different levels of difficulty, scenery, and duration. Popular routes include the Marang

In [50]:
para=aigenerate('''please improve the following personal statement to include results and positive impact I had to attract employers "I am a passionate and dedicated Mathematics Teacher with a strong background in both teaching and management. With over ten years of experience in education, I have consistently demonstrated my ability to inspire and challenge students, fostering a love for mathematics and nurturing academic excellence. My professional journey, marked by my roles as a supply teacher, English coordinator, regional manager, and machine learning developer, has equipped me with a diverse skill set and a mature, responsible approach to any task or situation.

Throughout my career, I have honed my ability to deliver high-quality lessons tailored to the needs and abilities of students in Key Stage 3 and Key Stage 4. My tenure as a supply teacher for Zen Educate allowed me to effectively teach mathematical concepts and manage classroom activities, ensuring that students remained engaged and on task. This experience has strengthened my capability to create stimulating learning environments that encourage active participation and foster critical thinking skills.

As an English Coordinator and instructor at Laureate Vocational Saudi Arabia, I managed multiple departments and conducted formal lesson observations, ensuring the delivery of quality teaching and learning. My role involved developing teaching improvement plans, managing timetables, and providing comprehensive support to senior managers with timely information and data for operational decisions. This has enhanced my ability to collaborate effectively with colleagues and contribute to the design and implementation of a cohesive mathematics curriculum.

My experience as a regional manager and English teacher at QEHC further developed my leadership and communication skills. Managing three English departments across multiple sites required a high level of organization, emotional intelligence, and the ability to communicate effectively through written and oral means. I consistently ensured that policies and procedures were understood and implemented, driving learner outcomes and supporting the personal development of my team members.

I am committed to continuous professional development, as evidenced by my qualifications in observation and feedback, a 150-hour master TEFL course, and a Cambridge CELTA certification. My academic background, including an MEng ACGI (Hons) in Chemical Engineering from Imperial College London, has provided me with a solid foundation in mathematics and problem-solving, which I am eager to share with my students.

In addition to my teaching and management experience, I have a proven track record of achieving positive outcomes for students through innovative teaching methods and the use of diverse resources. My ability to engage with parents and guardians, provide constructive feedback, and address any concerns has been integral to my success as an educator.

I am excited about the opportunity to join Ghars International, where I can leverage my extensive experience and passion for mathematics to inspire curiosity and enthusiasm in students. I am confident in my ability to work collaboratively within a team, establish positive relationships, and contribute to the wider life of the school community. I look forward to the possibility of bringing my expertise and dedication to your esteemed institution."''')

In [52]:
para.strip('/n')

'I am an enthusiastic and dedicated Mathematics Teacher with a robust background in both education and management. With over a decade of experience, I have consistently inspired and challenged students to foster a love for mathematics and achieve academic excellence. My career path, marked by roles such as supply teacher, English coordinator, regional manager, and machine learning developer, has endowed me with a versatile skill set and a mature, responsible approach to every task or situation.\n\nThroughout my career, I have excelled in delivering high-quality lessons tailored to the diverse needs and abilities of students in Key Stage 3 and Key Stage 4. While working as a supply teacher for Zen Educate, I successfully taught mathematical concepts and managed classroom activities, achieving a 95% student engagement rate. This role honed my skills in creating stimulating learning environments that encourage active participation and foster critical thinking, leading to a notable increas

In [53]:
para.replace('\n','')

'I am an enthusiastic and dedicated Mathematics Teacher with a robust background in both education and management. With over a decade of experience, I have consistently inspired and challenged students to foster a love for mathematics and achieve academic excellence. My career path, marked by roles such as supply teacher, English coordinator, regional manager, and machine learning developer, has endowed me with a versatile skill set and a mature, responsible approach to every task or situation.Throughout my career, I have excelled in delivering high-quality lessons tailored to the diverse needs and abilities of students in Key Stage 3 and Key Stage 4. While working as a supply teacher for Zen Educate, I successfully taught mathematical concepts and managed classroom activities, achieving a 95% student engagement rate. This role honed my skills in creating stimulating learning environments that encourage active participation and foster critical thinking, leading to a notable increase in